In [ ]:
import re

In [ ]:
expr = "2 * 3 + (4 * 5)"
expr = "5 + (8 * 3 + 9 + 3 * 4 * 3)"
expr = "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))"
expr = "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"

In [ ]:
filename = "day18.input"
with open(filename) as file:
    expressions = [line.strip() for line in file]

In [ ]:
def eval_match(match):
    return str(eval(match.group(0)))

# Part 1

In [ ]:
def compute(expr):

    # While there are operators left
    while re.search(r"[+*]", expr):
        
        # Replace the leftmost simple expression with its result
        expr = re.sub(r"\d+ [+*] \d+", eval_match, expr, count=1)

        # Replace a number inside () with just the number
        expr = re.sub(r"\((\d+)\)", r"\1", expr)
    
    return int(expr)

In [ ]:
sum(compute(expression) for expression in expressions)

# Part 2

In [ ]:
def compute2(expr):
    
    # While there are + operators left
    while re.search(r"\+", expr):
        # Replace simple additions with the evaluated result
        expr = re.sub(r"\d+ \+ \d+", eval_match, expr)
        
        # A () with only * in it, can be replaced with the result
        expr = re.sub(r"\((\d+( \* \d+)*)\)", eval_match, expr)
        
    # While there are * operators left
    while re.search(r"\*", expr):
        # Replace simple multiplication with the evaluated result
        expr = re.sub(r"\d+ \* \d+", eval_match, expr)

        # A () with just a number in it can be replaced by the number
        expr = re.sub(r"\((\d+)\)", r"\1", expr)

    return int(expr)

In [ ]:
sum(compute2(expression) for expression in expressions)

# Alternative solution - Shunting-yard algorithm

In [ ]:
import operator
import re

In [ ]:
filename = "day18.input"
with open(filename) as file:
    expressions = [line.strip() for line in file]

In [ ]:
def shunting_yard(expression):
    """Convert infix expression into RPN.
    
    Uses the Shunting-yard algorithm by Dijkstra:
    https://en.wikipedia.org/wiki/Shunting-yard_algorithm
    """    
    # Make the expression easier to tokenize by adding spaces
    expression = re.sub("([\(\)])", r" \1 ", expression)
    
    output, stack = [], []
    for token in expression.split():
        if re.match("\d+", token):
            output.append(token)
        elif token in operators:
            while stack \
                and (stack[-1] != "(") \
                and (precedence[stack[-1]] >= precedence[token]):
                    output.append(stack.pop())
            stack.append(token)
        elif token == "(":
            stack.append(token)
        elif token == ")":
            while stack:
                oper = stack.pop()
                if oper != "(":
                    output.append(oper)
                else:
                    break
    while stack:
        output.append(stack.pop())
        
    return output

In [ ]:
def rpn_eval(rpn):
    """Evaluate an RPN expression"""
    stack = []
    for token in rpn:
        if token in operators:
            arg1 = stack.pop()
            arg2 = stack.pop()
            stack.append(operators[token](arg1, arg2))
        else:
            stack.append(int(token))
    
    return stack.pop()

def compute(expression):
    return rpn_eval(shunting_yard(expression))

# Part 1

In [ ]:
precedence = {"+": 2, "*": 2}
operators = {"+": operator.add, "*": operator.mul}

In [ ]:
assert 26 == compute("2 * 3 + (4 * 5)")
assert 437 == compute("5 + (8 * 3 + 9 + 3 * 4 * 3)")
assert 12240 == compute("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))")
assert 13632 == compute("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2")

In [ ]:
sum(compute(expression) for expression in expressions)

# Part 2

In [ ]:
precedence = {"+": 3, "*": 2}
operators = {"+": operator.add, "*": operator.mul}

In [ ]:
assert 51 == compute("1 + (2 * 3) + (4 * (5 + 6))")
assert 46 == compute("2 * 3 + (4 * 5)")
assert 1445 == compute("5 + (8 * 3 + 9 + 3 * 4 * 3)")
assert 669060 == compute("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))")
assert 23340 == compute("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2")

In [ ]:
sum(compute(expression) for expression in expressions)